<a href="https://colab.research.google.com/github/priyanka-kumari-01/Extensive-Vision-AI-Program-EVA8/blob/main/S_2.5_pytorch_101/Session_2_5_PyTorch_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

In [ ]:
from torchvision.datasets.mnist import MNIST
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self):
       self.MNIST_Data = datasets.MNIST(
            root="data",
            train=True,
            download=True,
            transform=ToTensor()
        )
       self.rand_data = torch.randint(0,10,(1,len(self.MNIST_Data))).squeeze().to(device)

    def __getitem__(self, index):
        # Convert int to 1d array representing from 0 to 9
        random_array = [0,0,0,0,0,0,0,0,0,0]
        random_array[self.rand_data[index]]= 1
        image, label = self.MNIST_Data[index]
        return image, label, torch.tensor(random_array)

    def __len__(self):
        return len(self.MNIST_Data)


In [ ]:
myData = MyDataset()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
trainLoader = torch.utils.data.DataLoader(
    myData,
    batch_size=20,
    shuffle = True
)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        #Image: {in:28 out: 24} 
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        #Image: {in:24 out: 20} 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        #MaxPool: Image {in: 20 out: 10}
        #Image: {in:10 out: 8} 
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.fc1 = nn.Linear(in_features=128*8*8, out_features= 256)
        self.fc2 = nn.Linear(in_features=256, out_features= 10)
        self.out = nn.Linear(in_features=10, out_features= 19)

    def forward(self, img, rand):
        #x = x.reshape(1, 1*28*28)
        img = self.conv1(img)
        img = F.relu(img)
        img = self.conv2(img)
        img = F.relu(img)
        img = F.max_pool2d(img, kernel_size=2)
        img = self.conv3(img)
        img = F.relu(img)
        img = img.reshape(-1, 128*8*8)
        img = self.fc1(img)
        img = F.relu(img)
        preds_img = self.fc2(img)
        preds_randImg = preds_img + rand
        preds_randImg = self.out(preds_randImg)
        preds_randImg = F.softmax(preds_randImg, dim=1) #make result positive
        return preds_img, preds_randImg

network = Network()
network.to(device)

Network(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=8192, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
  (out): Linear(in_features=10, out_features=19, bias=True)
)

In [ ]:
for name, para in network.named_parameters():
    print(name, '\t\t', para.shape)

conv1.weight 		 torch.Size([32, 1, 5, 5])
conv1.bias 		 torch.Size([32])
conv2.weight 		 torch.Size([64, 32, 5, 5])
conv2.bias 		 torch.Size([64])
conv3.weight 		 torch.Size([128, 64, 3, 3])
conv3.bias 		 torch.Size([128])
fc1.weight 		 torch.Size([256, 8192])
fc1.bias 		 torch.Size([256])
fc2.weight 		 torch.Size([10, 256])
fc2.bias 		 torch.Size([10])
out.weight 		 torch.Size([19, 10])
out.bias 		 torch.Size([19])


In [ ]:
batch = next(iter(trainLoader))
image, label , random = batch
print(label)

tensor([3, 1, 2, 6, 4, 7, 7, 3, 5, 8, 9, 4, 6, 9, 7, 8, 1, 9, 4, 1])


In [ ]:
image, label , random = batch

In [ ]:
img_preds, preds_randImg  = network(image.to(device), random.to(device))
img_preds.shape, preds_randImg.shape

(torch.Size([20, 10]), torch.Size([20, 19]))

In [ ]:
random.argmax(dim=1)

tensor([7, 2, 1, 6, 2, 6, 1, 3, 6, 2, 0, 2, 2, 9, 6, 3, 1, 6, 4, 2])

In [ ]:
label, random.argmax(dim=1)

(tensor([3, 1, 2, 6, 4, 7, 7, 3, 5, 8, 9, 4, 6, 9, 7, 8, 1, 9, 4, 1]),
 tensor([7, 2, 1, 6, 2, 6, 1, 3, 6, 2, 0, 2, 2, 9, 6, 3, 1, 6, 4, 2]))

In [ ]:
#get_num_correct(preds_randImg, label+random.argmax(dim=1))

In [ ]:
def get_num_correct(pred , label):
    return pred.argmax(dim=1).eq(label).sum().item()

In [ ]:
optimizer = optim.Adam(network.parameters(), lr = 0.001)

for epoch in range(10):
    total_loss_img =0
    total_correct_img = 0
    total_loss_randImg =0
    total_correct_randImg= 0
    for batch in trainLoader:
        image, label , random = batch

        img_preds, preds_randImg = network(image.to(device), random.to(device))
        loss_img = F.cross_entropy(img_preds.to(device), label.to(device)) #image Output
        loss_randImg = F.cross_entropy(preds_randImg.to(device), (label+random.argmax(dim=1)).to(device))
        loss = loss_img+loss_randImg

        optimizer.zero_grad()
        loss_randImg.backward(retain_graph=True)
        loss_img.backward()
        #loss.backward()
        optimizer.step()

        total_loss_img+= loss_img.item()
        total_correct_img+= get_num_correct(img_preds.to(device), label.to(device))
        total_loss_randImg+= loss_randImg.item()
        total_correct_randImg+= get_num_correct(preds_randImg.to(device), (label+random.argmax(dim=1)).to(device))

    print(
        "epoch:", epoch,
        "mnist_correct: %.4f"%((total_correct_img/len(myData))*100),
        "mnistt_loss ", total_loss_img,
        "total_correct_randImg:  %.4f"%((total_correct_randImg/len(myData))*100),
        "total_loss_randImg ", total_loss_randImg,
        "total_correct: ", total_correct_img + total_correct_randImg,
        "total_loss %.4f"%(((total_loss_img+total_loss_randImg)/len(myData)*2)*100)
    )

epoch: 0 mnist_correct: 99.9800 mnistt_loss  5.147966181490119 total_correct_randImg:  40.7683 total_loss_randImg  7924.860264778137 total_correct:  84449 total_loss 26.4334
epoch: 1 mnist_correct: 99.9733 mnistt_loss  5.4595580078494095 total_correct_randImg:  41.8600 total_loss_randImg  7878.965674877167 total_correct:  85100 total_loss 26.2814
epoch: 2 mnist_correct: 99.9850 mnistt_loss  4.684863976076485 total_correct_randImg:  42.5850 total_loss_randImg  7847.921477794647 total_correct:  85542 total_loss 26.1754
epoch: 3 mnist_correct: 99.9750 mnistt_loss  4.423047744107862 total_correct_randImg:  43.0200 total_loss_randImg  7823.751688241959 total_correct:  85797 total_loss 26.0939
epoch: 4 mnist_correct: 99.9667 mnistt_loss  4.9244327028591215 total_correct_randImg:  43.3633 total_loss_randImg  7808.15487074852 total_correct:  85998 total_loss 26.0436
epoch: 5 mnist_correct: 99.9567 mnistt_loss  5.810167410459144 total_correct_randImg:  43.6700 total_loss_randImg  7794.111377477

In [ ]:
for epoch in range(40):
    total_loss_img =0
    total_correct_img = 0
    total_loss_randImg =0
    total_correct_randImg= 0
    for batch in trainLoader:
        image, label , random = batch

        img_preds, preds_randImg = network(image.to(device), random.to(device))
        loss_img = F.cross_entropy(img_preds.to(device), label.to(device)) #image Output
        loss_randImg = F.cross_entropy(preds_randImg.to(device), (label+random.argmax(dim=1)).to(device))
        loss = loss_img+loss_randImg

        optimizer.zero_grad()
        loss_randImg.backward(retain_graph=True)
        loss_img.backward()
        #loss.backward()
        optimizer.step()

        total_loss_img+= loss_img.item()
        total_correct_img+= get_num_correct(img_preds.to(device), label.to(device))
        total_loss_randImg+= loss_randImg.item()
        total_correct_randImg+= get_num_correct(preds_randImg.to(device), (label+random.argmax(dim=1)).to(device))

    print(
        "epoch:", epoch,
        "mnist_correct: %.4f"%((total_correct_img/len(myData))*100),
        "mnistt_loss ", total_loss_img,
        "total_correct_randImg:  %.4f"%((total_correct_randImg/len(myData))*100),
        "total_loss_randImg ", total_loss_randImg,
        "total_correct: ", total_correct_img + total_correct_randImg,
        "total_loss %.4f"%(((total_loss_img+total_loss_randImg)/len(myData)*2)*100)
    )

epoch: 0 mnist_correct: 99.9683 mnistt_loss  3.627679469161194 total_correct_randImg:  45.5000 total_loss_randImg  7730.380918979645 total_correct:  87281 total_loss 25.7800
epoch: 1 mnist_correct: 99.9717 mnistt_loss  3.8367421145924254 total_correct_randImg:  45.7133 total_loss_randImg  7722.177541971207 total_correct:  87411 total_loss 25.7534
epoch: 2 mnist_correct: 99.9750 mnistt_loss  3.457020449563288 total_correct_randImg:  45.8200 total_loss_randImg  7718.915107011795 total_correct:  87477 total_loss 25.7412
epoch: 3 mnist_correct: 99.9783 mnistt_loss  3.152484122651458 total_correct_randImg:  46.0467 total_loss_randImg  7709.893150806427 total_correct:  87615 total_loss 25.7102
epoch: 4 mnist_correct: 99.9850 mnistt_loss  3.078210401255319 total_correct_randImg:  46.0967 total_loss_randImg  7707.625150680542 total_correct:  87649 total_loss 25.7023
epoch: 5 mnist_correct: 99.9783 mnistt_loss  3.5733740038569977 total_correct_randImg:  46.2800 total_loss_randImg  7701.05273389